In [ ]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("../Resources/exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Display dataframe
df.head()

# Select features and label

In [3]:
# Set features as X value and label as y value
X = df.drop(columns=['koi_disposition'])
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [5]:
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

# Create Model

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
model = RandomForestClassifier(random_state = 1)

# Fit (train) or model using the training data
model = model.fit(X_train_scaled, y_train)

In [7]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8941647597254004


In [8]:
# print prediction results
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions)) 

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.74      0.78       422
     CONFIRMED       0.80      0.84      0.82       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Identify insignificant variables

In [9]:
# Sort the features by their importance
features_imp = pd.DataFrame({
    'features': X.columns,
    'probability': model.feature_importances_
})

features_imp = features_imp.sort_values('probability', ascending=False)

features_imp.head()

,features,probability
2,koi_fpflag_co,0.107540
0,koi_fpflag_nt,0.104147
1,koi_fpflag_ss,0.073996
26,koi_model_snr,0.057301
19,koi_prad,0.052542


In [10]:
# Locate features with probability > 0.01
selected_features_df = features_imp.loc[features_imp['probability'] > 0.01]

# Get features based on above condition
selected_features = list(selected_features_df['features'])

selected_features

['koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad',
 'koi_duration_err2',
 'koi_fpflag_ec',
 'koi_prad_err2',
 'koi_steff_err1',
 'koi_duration_err1',
 'koi_duration',
 'koi_period',
 'koi_prad_err1',
 'koi_time0bk_err1',
 'koi_depth',
 'koi_steff_err2',
 'koi_time0bk_err2',
 'koi_period_err2',
 'koi_insol_err1',
 'koi_period_err1',
 'koi_impact',
 'koi_teq',
 'koi_time0bk',
 'koi_depth_err1',
 'koi_depth_err2',
 'dec',
 'koi_insol',
 'ra',
 'koi_impact_err1',
 'koi_insol_err2',
 'koi_impact_err2',
 'koi_slogg']

# Train Model (with selected features)

In [11]:
# Set features as X value and label as y value
X = df[selected_features]
y = df['koi_disposition']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

In [14]:
# Create a random forest classifier
model = RandomForestClassifier(random_state = 1)

# Fit (train) or model using the training data
model = model.fit(X_train_scaled, y_train)

In [15]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8913043478260869


In [16]:
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions)) 

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.73      0.78       422
     CONFIRMED       0.79      0.84      0.81       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.86      0.85      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import FitFailedWarning
import warnings
warnings.simplefilter('always', FitFailedWarning)

# Create the GridSearchCV model: https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

# Defining parameter range 
param_grid = {
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10] ,
    'min_samples_split': [2, 5, 10, 15, 100],
    'n_estimators': [100, 300, 500, 800, 1200]
}
  
grid = GridSearchCV(model, param_grid, cv = 3, n_jobs = -1, verbose = 1) 

In [18]:
# Source: https://scikit-learn.org/0.22/modules/generated/sklearn.exceptions.FitFailedWarning.html
# Fit the model using the grid search estimator.
with warnings.catch_warnings(record=True) as w:
    try:
        grid.fit(X_train_scaled, y_train)
    except ValueError:
         pass

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


In [19]:
print(f'Best Parameter: {grid.best_params_}')
print(f'Best Model Score: {grid.best_score_}')

Best Parameter: {'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Best Model Score: 0.8983419107485995


In [20]:
predictions = grid.predict(X_test_scaled)
print(classification_report(y_test, predictions)) 

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.75      0.79       422
     CONFIRMED       0.80      0.84      0.82       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Save the Model

In [21]:
import joblib
from joblib import dump, load

# save model
filename = '../Models/decision_tree.sav'
joblib.dump(grid, filename)

['../Models/decision_tree.sav']